Предложить метод выбора наиболее важных признаков для логистической регрессии на основе изученных методов прикладной статистики. Осуществить выбор.

In [2]:
import eli5
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.feature_selection import chi2, SelectPercentile
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [3]:
data = load_wine()
X, y, feature_names = data['data'], data['target'], data['feature_names']

Воспользуемся критерием проверки независимости `target` с каждым из признаков. Тестовая статистика будет иметь распределение хи-квадрат. Признаки с большим p-value более вероятно будут независимы с `target`, поэтому можем отсортировать по возрастанию p-value и взять топ n.

In [4]:
selector = SelectPercentile(chi2, percentile=0)
selector.fit(X, y)
pd.DataFrame({'feature': feature_names, 'score': selector.scores_, 'p-value': selector.pvalues_}).sort_values('p-value')

,feature,score,p-value
12,proline,16540.067145,0.000000e+00
9,color_intensity,109.016647,2.124887e-24
6,flavanoids,63.334308,1.766565e-14
4,magnesium,45.026381,1.669728e-10
3,alcalinity_of_ash,29.383695,4.163050e-07
1,malic_acid,28.068605,8.034890e-07
11,od280/od315_of_diluted_wines,23.389883,8.335878e-06
5,total_phenols,15.623076,4.050346e-04
8,proanthocyanins,9.368283,9.240664e-03
0,alcohol,5.445499,6.569389e-02


Можно сравнить с более популярным на практике методом

In [5]:
lr = LogisticRegression(penalty='l1', random_state=16, solver='liblinear')
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
lr.fit(X_scaled, y)
eli5.explain_weights(lr, feature_names=feature_names)

Explanation(estimator="LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=100,\n                   multi_class='auto', n_jobs=None, penalty='l1',\n                   random_state=16, solver='liblinear', tol=0.0001, verbose=0,\n                   warm_start=False)", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=0, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='proline', weight=2.467493273750969, std=None, value=None), FeatureWeight(feature='alcohol', weight=1.5101758473926818, std=None, value=None), FeatureWeight(feature='od280/od315_of_diluted_wines', weight=1.186625392975417, std=None, value=None), FeatureWeight(feature='ash', weight=0.9623054009347871, std=None, value=None), FeatureWeight(feature='flavanoids', weight=0.9096630658451123, std=None, value=None), FeatureWeight(feature='malic_acid', weight=0.2540824894462918, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-1.4911577408394538, std=None, value=None), FeatureWeight(feature='alcalinity_of_ash', weight=-1.4759549035782538, std=None, value=None)], pos_remaining=0, neg_remaining=0), proba=None, score=None, weighted_spans=None, heatmap=None), TargetExplanation(target=1, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='hue', weight=1.2392693878467735, std=None, value=None), FeatureWeight(feature='alcalinity_of_ash', weight=0.5681368961523539, std=None, value=None), FeatureWeight(feature='flavanoids', weight=0.5007839858855152, std=None, value=None), FeatureWeight(feature='nonflavanoid_phenols', weight=0.20348308185388234, std=None, value=None), FeatureWeight(feature='proanthocyanins', weight=0.13514278541674468, std=None, value=None)], neg=[FeatureWeight(feature='proline', weight=-2.4813782373707287, std=None, value=None), FeatureWeight(feature='color_intensity', weight=-2.3151444278263176, std=None, value=None), FeatureWeight(feature='alcohol', weight=-1.598860496721543, std=None, value=None), FeatureWeight(feature='<BIAS>', weight=-1.5523254281385377, std=None, value=None), FeatureWeight(feature='ash', weight=-1.1175558391089782, std=None, value=None), FeatureWeight(feature='malic_acid', weight=-0.5524939595376946, std=None, value=None)], pos_remaining=0, neg_remaining=0), proba=None, score=None, weighted_spans=None, heatmap=None), TargetExplanation(target=2, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='color_intensity', weight=1.950672994287947, std=None, value=None), FeatureWeight(feature='ash', weight=0.5059545059819432, std=None, value=None), FeatureWeight(feature='malic_acid', weight=0.2519155294758307, std=None, value=None), FeatureWeight(feature='alcohol', weight=0.04911448666683635, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-2.4978261427094455, std=None, value=None), FeatureWeight(feature='flavanoids', weight=-2.4291736594258704, std=None, value=None), FeatureWeight(feature='hue', weight=-1.111414923398842, std=None, value=None), FeatureWeight(feature='od280/od315_of_diluted_wines', weight=-0.6139669026894531, std=None, value=None), FeatureWeight(feature='nonflavanoid_phenols', weight=-0.055249656504139065, std=None, value=None)], pos_remaining=0, neg_remaining=0), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [6]:
lr.coef_

array([[ 1.51017585,  0.25408249,  0.9623054 , -1.4759549 ,  0.        ,
         0.        ,  0.90966307,  0.        ,  0.        ,  0.        ,
         0.        ,  1.18662539,  2.46749327],
       [-1.5988605 , -0.55249396, -1.11755584,  0.5681369 ,  0.        ,
         0.        ,  0.50078399,  0.20348308,  0.13514279, -2.31514443,
         1.23926939,  0.        , -2.48137824],
       [ 0.04911449,  0.25191553,  0.50595451,  0.        ,  0.        ,
         0.        , -2.42917366, -0.05524966,  0.        ,  1.95067299,
        -1.11141492, -0.6139669 ,  0.        ]])

Результаты в целом согласуются, кроме, быть может,  признака `magnesium`. Насколько я понял, он связан с `proline`, видимо из-за этого лассо понизила его влияние.